In [1]:
import os
import pandas as pd
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.stats import zscore
from datetime import datetime, timedelta
import neurokit2 as nk
from scipy.signal import welch

In [2]:
def timedomain(rr):
    """Calculate time domain HRV metrics from RR intervals."""
    results = {}
    if len(rr) == 0:
        # If rr is empty, return NaNs or some default values
        results['Mean RR (ms)'] = np.nan
        results['STD RR/SDNN (ms)'] = np.nan
        results['RMSSD (ms)'] = np.nan
        results['Mean HR (Kubios\' style) (beats/min)'] = np.nan
        results['Mean HR (beats/min)'] = np.nan
        results['STD HR (beats/min)'] = np.nan
        results['Min HR (beats/min)'] = np.nan
        results['Max HR (beats/min)'] = np.nan
        # results['NN50'] = np.nan
        # results['pNN50 (%)'] = np.nan
    else:
        hr = 60 / rr
        results['Mean RR (ms)'] = np.mean(rr)
        results['STD RR/SDNN (ms)'] = np.std(rr)
        results['RMSSD (ms)'] = np.sqrt(np.mean(np.square(np.diff(rr))))
        results['Mean HR (Kubios\' style) (beats/min)'] = 60000 / np.mean(rr)
        results['Mean HR (beats/min)'] = np.mean(hr)
        results['STD HR (beats/min)'] = np.std(hr)
        results['Min HR (beats/min)'] = np.min(hr)
        results['Max HR (beats/min)'] = np.max(hr)
        # results['NN50'] = np.sum(np.abs(np.diff(rr)) > 50) * 1
        # results['pNN50 (%)'] = 100 * np.sum((np.abs(np.diff(rr)) > 50) * 1) / len(rr)
    return results


In [3]:
def AbsolutePower(signal, fs, low, high):
    """Calculate absolute power in a specific frequency band."""
    f, Pxx = welch(signal, fs, nperseg=256)
    band_power = np.trapz(Pxx[(f >= low) & (f <= high)], f[(f >= low) & (f <= high)])
    return band_power

In [4]:
def RelativePower(signal, fs, low, high):
    """Calculate relative power in a specific frequency band."""
    total_power = AbsolutePower(signal, fs, 0, fs / 2)
    band_power = AbsolutePower(signal, fs, low, high)
    return band_power / total_power

In [5]:
def capture_uco_windows_ecg(ecg_data, fs, start_time, uco_start_time, uco_end_time, additional_file_before=None, additional_start_time_before=None, additional_file_after=None, additional_start_time_after=None, window_len_sec=15):
    """
    Capture UCO time before and after windows and compute ECG features.

    Parameters
    ----------
    ecg_data : array-like
        The ECG signal data.
    fs : int
        Sampling frequency of the ECG data.
    start_time : str
        Start time of the ECG data in format 'HH:MM:SS:%f %p'.
    uco_start_time : str
        UCO start time in format 'HH:MM:SS.%f %p'.
    uco_end_time : str
        UCO end time in format 'HH:MM:SS.%f %p'.
    additional_file_before : str, optional
        Path to the additional file needed for segments before UCO start time.
    additional_start_time_before : str, optional
        Start time of the additional file for segments before UCO start time.
    additional_file_after : str, optional
        Path to the additional file needed for segments after UCO end time.
    additional_start_time_after : str, optional
        Start time of the additional file for segments after UCO end time.
    window_len_sec : int
        Length of each window in seconds.

    Returns
    -------
    df : DataFrame
        DataFrame containing computed features for each window.
    """
    # Convert times to datetime objects
    start_datetime = datetime.strptime(start_time, '%I:%M:%S:%f %p')
    uco_start_datetime = datetime.strptime(uco_start_time, '%I:%M:%S.%f %p')
    uco_end_datetime = datetime.strptime(uco_end_time, '%I:%M:%S.%f %p')

    # Adjust for UCO times that cross midnight
    if uco_start_datetime < start_datetime:
        uco_start_datetime += timedelta(days=1)
    if uco_end_datetime < uco_start_datetime:
        uco_end_datetime += timedelta(days=1)
    
    # Calculate the difference in seconds between the start time and UCO times
    time_diff_start = (uco_start_datetime - start_datetime).total_seconds()
    time_diff_end = (uco_end_datetime - start_datetime).total_seconds()
    
    # Convert the time differences to sample indices
    uco_start_samples = int(time_diff_start * fs)
    uco_end_samples = int(time_diff_end * fs)
    
    # Define window length in samples
    window_len = window_len_sec * fs
    
    # Calculate the start and end sample indices for the desired windows
    start_sample_before = max(0, uco_start_samples - 75 * 60 * fs)
    end_sample_before = uco_start_samples - 60 * 60 * fs
    start_sample_uco = uco_start_samples
    end_sample_uco = uco_end_samples
    start_sample_after = uco_end_samples
    end_sample_after = min(len(ecg_data), uco_end_samples + 60 * window_len)

    print(f"time diff start: {time_diff_start}")
    print(f"time diff end: {time_diff_end}")
    print(f"start sample before UCO: {start_sample_before}")
    print(f"end sample before UCO: {end_sample_before}")
    print(f"start sample after UCO: {start_sample_after}")
    print(f"end sample after UCO: {end_sample_after}")

    data = {
        'time': [],
        'sdrr': [],
        'rmssd': [],
        'mrr': [],
        'mean_hr': [],
        'std_hr': [],
        'min_hr': [],
        'max_hr': [],
        'label': []
    }

    def process_windows(signal_data, start_sample, end_sample, label):
        for start in range(start_sample, end_sample, window_len):
            end = start + window_len
            if end > len(signal_data):
                break  # Ensure the last window doesn't exceed the data length
    
            segment = signal_data[start:end]
            rr_peaks, _ = find_peaks(segment)
            rr_intervals = np.diff(rr_peaks)
            rr_intervals[np.abs(zscore(rr_intervals)) > 2] = np.median(rr_intervals)
    
            if len(rr_intervals) == 0:
                print(f"No peaks detected in segment starting at {start / fs} seconds.")
                continue  # Skip this segment if no peaks are detected
    
            hrv_metrics = timedomain(rr_intervals)
    
            # Append data to the dictionary
            data['time'].append(start / fs)
            data['sdrr'].append(hrv_metrics['STD RR/SDNN (ms)'])
            data['rmssd'].append(hrv_metrics['RMSSD (ms)'])
            data['mrr'].append(hrv_metrics['Mean RR (ms)'])
            data['mean_hr'].append(hrv_metrics['Mean HR (beats/min)'])
            data['std_hr'].append(hrv_metrics['STD HR (beats/min)'])
            data['min_hr'].append(hrv_metrics['Min HR (beats/min)'])
            data['max_hr'].append(hrv_metrics['Max HR (beats/min)'])
            # data['nn50'].append(hrv_metrics['NN50'])
            # data['pnn50'].append(hrv_metrics['pNN50 (%)'])
            # data['absolute_power_low'].append(absolute_power_low)
            # data['absolute_power_high'].append(absolute_power_high)
            # data['relative_power_low'].append(relative_power_low)
            # data['relative_power_high'].append(relative_power_high)
            data['label'].append(label)
    
            plt.close()


    # Process windows 1 hour and 15 minutes to 1 hour before UCO start time and label as -1
    if start_sample_before < end_sample_before:
        process_windows(ecg_data, start_sample_before, end_sample_before, label=-1)
    elif additional_file_before and additional_start_time_before:
        additional_data = loadmat(additional_file_before)['save_data'].flatten()
        new_start_datetime_before = datetime.strptime(additional_start_time_before, '%I:%M:%S:%f %p')
        
        # Adjust for additional start time crossing midnight
        if uco_start_datetime < new_start_datetime_before:
            uco_start_datetime += timedelta(days=1)
        
        additional_time_diff_start = (uco_start_datetime - new_start_datetime_before).total_seconds() - 75 * 60
        additional_start_samples = int(additional_time_diff_start * fs)
        process_windows(additional_data, additional_start_samples, additional_start_samples + 60 * window_len, label=-1)
    
    # Process windows during UCO and label as 0
    process_windows(ecg_data, start_sample_uco, end_sample_uco, label=0)
    
    # Process windows after UCO end time and label as 1
    remaining_duration = len(ecg_data) - start_sample_after
    if remaining_duration >= 15 * 60 * fs:
        process_windows(ecg_data, start_sample_after, end_sample_after, label=1)
    else:
        process_windows(ecg_data, start_sample_after, len(ecg_data), label=1)
        if additional_file_after and additional_start_time_after:
            additional_data = loadmat(additional_file_after)['save_data'].flatten()
            new_start_datetime_after = datetime.strptime(additional_start_time_after, '%I:%M:%S:%f %p')
            
            # Adjust for additional start time crossing midnight
            if new_start_datetime_after < start_datetime:
                new_start_datetime_after += timedelta(days=1)
            
            additional_start_samples = 0  # Start from the beginning of the additional file
            additional_end_samples = int((15 * 60 * fs - remaining_duration))
            process_windows(additional_data, additional_start_samples, additional_end_samples, label=1)

    df = pd.DataFrame(data)
    return df

In [6]:
# Define the parameters
main_file = r'D:\IEEE Sensor\IEEE Sensor Matlab Data\ECG\21044\Clean Data ECGCUsersvhor0002DownloadsCODE BACK UP-20240215T015228Z-001CODE BACK UPRaw Data2104421044_1-06-2021 9_25_16.2 AM_ UCO file Channel 2.mat'
handled_folder_path = r'D:\IEEE Sensor\IEEE Sensor Xlsx Data\ECG_handled'
fs = 400  # Sampling frequency
start_time = '09:25:16:2 AM'  # Start time of the data
uco_start_time = '09:54:34.055758 AM'  # UCO start time
uco_end_time = '10:09:31.555758 AM'  # UCO end time

additional_file_before = r'D:\IEEE Sensor\IEEE Sensor Matlab Data\ECG\21044\Clean Data ECGCUsersvhor0002DownloadsCODE BACK UP-20240215T015228Z-001CODE BACK UPRaw Data2104421044_1-06-2021 9_25_16.2 AM_ UCO file Channel 1.mat'  # Path to additional file for before UCO
additional_start_time_before = '11:52:09:0 PM'  # Start time of the additional file for before UCO
additional_file_after = r'D:\IEEE Sensor\IEEE Sensor Matlab Data\ECG\21126\t'  # Path to additional file for after UCO
additional_start_time_after = '09:05:07:3 AM' # Start time of the additional file for after UCO

# Make sure the processed folder exists
if not os.path.exists(handled_folder_path):
    os.makedirs(handled_folder_path)

# Load the main file
main_data = loadmat(main_file)
ecg_data = main_data['save_data'].flatten()

# Capture UCO windows and compute features
df = capture_uco_windows_ecg(ecg_data, fs, start_time, uco_start_time, uco_end_time, additional_file_before, additional_start_time_before, additional_file_after, additional_start_time_after)

# Define new save path
folder_name = os.path.basename(os.path.dirname(main_file))
new_save_folder = os.path.join(handled_folder_path, folder_name + '_handled')
if not os.path.exists(new_save_folder):
    os.makedirs(new_save_folder)

excel_file_name = os.path.basename(main_file)[:-4] + '.xlsx'  # Change the extension to .xlsx
new_excel_file_path = os.path.join(new_save_folder, excel_file_name)

df.to_excel(new_excel_file_path, index=False)

print("Processing completed! All processed files have been saved.")

time diff start: 1757.855758
time diff end: 2655.355758
start sample before UCO: 0
end sample before UCO: -736858
start sample after UCO: 1062142
end sample after UCO: 1422142
Processing completed! All processed files have been saved.
